# Use [GeoV](https://github.com/geov-ai/geov)-9b model with Huggingface transformers library

This notebook shows how to load and use [GeoV](https://github.com/geov-ai/geov)-9b model with Huggingface transformers library.

The model weights are on huggingface hub: [huggingface.co/GeoV/GeoV-9b](https://huggingface.co/GeoV/GeoV-9b)

### Install dependencies

In [ ]:
%%capture

!pip install transformers accelerate bitsandbytes sentencepiece

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

## Generate text with `transformers.pipeline`

### Create the pipeline

Note that you need to specify `trust_remote_code=True` since `GeoV` is not integrated into the transformers library.
It will download the code from the hub ([huggingface.co/GeoV/GeoV-9b](https://huggingface.co/GeoV/GeoV-9b)) and execute.

`load_in_8bit` will load the models weights in 8-bits and use [LLM.int8()](https://papers.labml.ai/paper/eb2bcaee1d0011edaa66a71c10a887e7), enabling 
the model to fit 12GB memory.

In [ ]:
generator = pipeline(model="GeoV/GeoV-9b", trust_remote_code=True, device_map='auto', model_kwargs={"load_in_8bit": True})

### Generate

In [ ]:
with torch.no_grad():
    res = generator("The best way to prepare for the everest basecamp trek is", do_sample=False, max_length=64)

res

## Load the model and tokenizer separately

In [ ]:
model = AutoModelForCausalLM.from_pretrained('GeoV/GeoV-9b', load_in_8bit=True, device_map='auto', trust_remote_code=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('GeoV/GeoV-9b', trust_remote_code=True)

In [ ]:
prompt = "In mathematics, topology is the study of"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

In [ ]:
with torch.no_grad():
    gen_tokens = model.generate(
        input_ids=input_ids.to(torch.device('cuda:0')),
        do_sample=True,
        temperature=1.,
        max_length=256,
    )

In [ ]:
tokenizer.batch_decode(gen_tokens)[0]